In [ ]:
!wget 'https://anaconda.org/conda-forge/libjpeg-turbo/2.1.0/download/linux-64/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -q
!wget 'https://anaconda.org/conda-forge/libgcc-ng/9.3.0/download/linux-64/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -q
!wget 'https://anaconda.org/conda-forge/gdcm/2.8.9/download/linux-64/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -q
!wget 'https://anaconda.org/conda-forge/conda/4.10.1/download/linux-64/conda-4.10.1-py37h89c1867_0.tar.bz2' -q
!wget 'https://anaconda.org/conda-forge/certifi/2020.12.5/download/linux-64/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -q
!wget 'https://anaconda.org/conda-forge/openssl/1.1.1k/download/linux-64/openssl-1.1.1k-h7f98852_0.tar.bz2' -q

!conda install 'libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install 'libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install 'gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install 'conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install 'certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install 'openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
import os

from PIL import Image
import pandas as pd
train = pd.read_csv('/kaggle/input/siim-covid19-detection/train_image_level.csv')
train_study = pd.read_csv('/kaggle/input/siim-covid19-detection/train_study_level.csv')
train_study['StudyInstanceUID'] = train_study['id'].apply(lambda x: x.replace('_study', ''))
del train_study['id']
train = train.merge(train_study, on='StudyInstanceUID')
class_names = ['Negative for Pneumonia','Typical Appearance', 'Indeterminate Appearance','Atypical Appearance']
classMap={0:[1, 0, 0, 0],
         1:[0, 1, 0, 0],
         2:[0, 0, 1, 0],
         3:[0, 0, 0, 1]}

def findClass(value):
    value=value.tolist()
    for key,val in classMap.items():
        if value==val:
            return key
train["class"] =[findClass(li) for li in train[class_names].values]


In [ ]:
imgPath_df=pd.DataFrame(columns=['id','path'])
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/train'):
    for filename in filenames:
        imgPath=os.path.join(dirname, filename)
        imgPath_df=imgPath_df.append({'id':filename.replace('.dcm','_image'),'path':imgPath},ignore_index=True)


train = train.merge(imgPath_df, on='id')
train.to_csv('/kaggle/working/train.csv',index=False)

In [ ]:
train.head()

https://www.kaggle.com/jayanth0572/inference-studyclass-baseline/edit

In [ ]:

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import numpy as np

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
from torch import Tensor
from torch.utils.data import Dataset
from torchvision import transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
import numpy as np
import torch
from torch import nn
from torch import optim
from numpy import asarray

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, transform=None, target_transform=None):
        
        self.img_labels=pd.read_csv(annotations_file)[:1000]
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path =self.img_labels.iloc[idx,9]
        image=read_xray(img_path)
        image=resize(image, 416)
        image=asarray(image)
        image = torch.from_numpy(image).long()
        image=np.stack((image,)*3, axis=0)
        label = self.img_labels.iloc[idx, 8]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        
        return image, label
    

https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/

https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model = models.resnet50(pretrained=False)
for param in model.parameters():
    param.requires_grad = True
    
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 4),
                                 nn.Softmax(dim=1))

model.to(device)

In [ ]:
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion =nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    # enumerate epochs
    for epoch in range(5):
        # enumerate mini batches
        for inputs, labels in train_dl:
            inputs, labels = inputs.to(device), labels.to(device)
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            logps = model.forward(inputs.float())
            # calculate loss
            loss = criterion(logps, labels.long())
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

In [ ]:
image_dataset = CustomImageDataset(annotations_file='/kaggle/working/train.csv')
train_dataloader = DataLoader(image_dataset, batch_size=10, shuffle=True)
train_model(train_dataloader,model)
torch.save(model, 'model.pth')

In [ ]:
model.eval()
col=['id','PredictionString']
test_transforms = transforms.Compose([transforms.ToTensor(),
                                     ])
prediction=pd.DataFrame(columns = col) 
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/test'):
    for filename in filenames:
        img_path= os.path.join(dirname, filename)
        id_=img_path.split('/')[5]+"_study"
        image=read_xray(img_path)
        image=resize(image, 416)
        image=asarray(image)
        image = torch.from_numpy(image).long()
        image=np.stack((image,)*3, axis=0)
        image_tensor = test_transforms(image).float()
        image_tensor = image_tensor.unsqueeze_(0)
        #inputs = Variable(image_tensor)
        inputs = image_tensor.to(device)
        inputs=torch.transpose(inputs, 1, 2)
        output = model(inputs)
        predProb = output.detach().cpu().numpy()[0]
        PredictionString=f'negative {format(predProb[0],".8f")} 0 0 1 1 typical {format(predProb[1],".5f")} 0 0 1 1 indeterminate {format(predProb[2],".5f")} 0 0 1 1 atypical {format(predProb[3],".5f")} 0 0 1 1' 
        #print(PredictionString)
        prediction=prediction.append({"id":id_,"PredictionString":PredictionString},ignore_index=True)

In [ ]:
def CreateSub(test):
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    for i in range(len(sub_df)):
        #print(test[test['id']==sub_df.loc[i,'id']])
        #negative, typical, indeterminate, atypical = str(tst_preds[i][0]),str(tst_preds[i][1]),str(tst_preds[i][2]),str(tst_preds[i][3]),
        try:
            sub_df.loc[i,'PredictionString'] =test[test['id']==sub_df.loc[i,'id']].iloc[0,1]# test[test['id']==sub_df.loc[i,'id']][0][1]#f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'
        except:
            continue
    return sub_df

In [ ]:
sumfile = CreateSub(prediction)
sumfile.to_csv('./submission.csv',index=False)